# L: Data Lake and Warehouse

In [1]:
import os
from datetime import datetime
import pandas as pd
import requests
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup

## Web Scraping

In [2]:
STORAGE_DIR = "local"

In [3]:
# List of news pages to be scraped
newspaper_urls = dict(
    sz="https://www.sueddeutsche.de/",
    zeit="https://www.zeit.de/index",
    faz="https://www.faz.net/aktuell/",
    ts="https://www.tagesspiegel.de/",
    spiegel="https://www.spiegel.de/",
    kronen="https://www.krone.at/",
    wtf="https://asdfkajwlkejwkejklajsdflksadjfasdf.nix",
)

In [4]:
# Current date as string
now = datetime.now()
now_str = now.strftime("%Y-%m-%d")
print(now_str)

2021-03-23


In [5]:
content_dict = {}
text_dict = {}
log_list = []
failing_list = []

In [6]:
def scrape_website(name, url):
    
    # (1) Run request
    response = requests.get(url, allow_redirects=True)
    content = response.content
    text = response.text
    
    # (2) File name to store the raw HTML
    file_name = os.path.join(
        STORAGE_DIR,
        f"{now_str}-{name}.html",
    )
    
    # (3) Write raw HTML
    with open(file_name, "wb") as f:
        f.write(response.content)
        
    # (4) Fill content_dict and text_dict
    content_dict[name] = response.content
    text_dict[name] = response.text
    
    # (5) Fill log_list
    log_info = dict(
        name=name,
        date=now_str,
        file_name=file_name,
        status=response.status_code,
        url=response.url,
        encoding=response.encoding,
    )
    log_list.append(log_info)

In [7]:
for name, url in newspaper_urls.items():
    try:
        scrape_website(name, url)
    except:
        failing_list.append((name, url))

In [8]:
log_df = pd.DataFrame(log_list)
log_df

,name,date,file_name,status,url,encoding
0,sz,2021-03-23,local\2021-03-23-sz.html,200,https://www.sueddeutsche.de/,UTF-8
1,zeit,2021-03-23,local\2021-03-23-zeit.html,200,https://www.zeit.de/index,UTF-8
2,faz,2021-03-23,local\2021-03-23-faz.html,200,https://www.faz.net/aktuell/,utf-8
3,ts,2021-03-23,local\2021-03-23-ts.html,200,https://www.tagesspiegel.de/,utf-8
4,spiegel,2021-03-23,local\2021-03-23-spiegel.html,200,https://www.spiegel.de/,utf-8
5,kronen,2021-03-23,local\2021-03-23-kronen.html,200,https://www.krone.at/,ISO-8859-1


In [9]:
log_file_name = os.path.join(
    STORAGE_DIR,
    f"{now_str}.csv",
)

In [10]:
log_df.to_csv(log_file_name)

## Word Count

In [11]:
stopwords_url = "https://raw.githubusercontent.com/solariz/german_stopwords/master/german_stopwords_full.txt"
stopwords_list = requests.get(stopwords_url, allow_redirects=True).text.split("\n")[9:]

In [12]:
def process_html(text):
    text = BeautifulSoup(text, "html.parser").text
    items = text.replace("\n", " ").lower().split(" ")
    items = [i for i in items if len(i) > 1 and i not in stopwords_list]
    return items

In [13]:
data = pd.DataFrame()
for name, text in text_dict.items():
    items = process_html(text)
    new = pd.Series(items).value_counts().to_frame()
    new.columns = ["count"]
    new["word"] = new.index
    new["name"] = name
    new["date"] = now_str
    data = pd.concat([data, new])

In [14]:
data

,count,word,name,date
bilder,82,bilder,sz,2021-03-23
sz,59,sz,sz,2021-03-23
gutschein,40,gutschein,sz,2021-03-23
plus,33,plus,sz,2021-03-23
momentaufnahmen,22,momentaufnahmen,sz,2021-03-23
...,...,...,...,...
geben!unterhaus,1,geben!unterhaus,kronen,2021-03-23
warmâkroneâ-wetter-updatewoche,1,warmâkroneâ-wetter-updatewoche,kronen,2021-03-23
pernsteiner,1,pernsteiner,kronen,2021-03-23
kã¼chenmaschine,1,kã¼chenmaschine,kronen,2021-03-23


In [15]:
data.loc[data["word"].str.contains("impfung"),]

,count,word,name,date
impfungen,1,impfungen,sz,2021-03-23
corona-impfung,2,corona-impfung,zeit,2021-03-23
corona-impfung:,1,corona-impfung:,zeit,2021-03-23
impfungen,1,impfungen,zeit,2021-03-23
corona-impfungen,1,corona-impfungen,zeit,2021-03-23
\t\t\t\t\t\t\t\t\t\t\t\timpfungen,1,\t\t\t\t\t\t\t\t\t\t\t\timpfungen,faz,2021-03-23
astra-zeneca-impfung,1,astra-zeneca-impfung,faz,2021-03-23
impfungen,2,impfungen,ts,2021-03-23
impfung,2,impfung,ts,2021-03-23
corona-impfung,1,corona-impfung,ts,2021-03-23


## SQLite

In [16]:
SQL_PATH = os.path.join("dwh.sqlite3")

In [17]:
connection = sqlite3.connect(SQL_PATH)
data.to_sql("wordcount", connection, index=False, if_exists="append")